# Amebaブログ　ダイエット記録

### ヘッダーとプロフィール欄に「摂食障害」，「拒食症」，「過食症」を含むユーザを除外

In [20]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import time

max_pages = 2  # 取得するページ数
max_blog_pages = 1  # 各ブログから取得するページ数
exclude_keywords = ["摂食障害", "拒食症", "過食症"]

from tqdm import tqdm

def scrape_ranking_items(url, max_pages):
    base_url = 'https://blogger.ameba.jp'
    items_data = []

    for page in range(max_pages):
        print(f"ページ {page + 1} を取得中...")
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # ランキングリストの取得
        ranking_list = soup.find('ol', class_='p-rankingAllImage')
        if not ranking_list:
            print("ランキングリストが見つかりませんでした。")
            break

        # ランキングアイテムの抽出
        ranking_items = ranking_list.find_all('li', class_='p-rankingAllText__item')

        # tqdmで進捗バーを表示しながらループ
        for item in tqdm(ranking_items, desc="ブログ取得中"):
            title_tag = item.find('h3', class_='p-rankingAllText__title').find('a')
            title = title_tag.get_text(strip=True) if title_tag else 'N/A'
            link = title_tag['href'] if title_tag and 'href' in title_tag.attrs else 'N/A'

            # ヘッダーとプロフィール情報の取得
            if not contains_excluded_keywords(link):
                # 各ブログの情報を取得（ブログ本文と日付とプロフィール情報を含む）
                blog_series_info = scrape_blog_series(title, link, max_blog_pages)
                items_data.extend(blog_series_info)

        # 次のページのリンクを取得
        next_link_tag = soup.find('li', class_='c-pager__item--next').find('a')
        if not next_link_tag:
            print("次のページが見つかりませんでした。")
            break
        next_link = base_url + next_link_tag['href']

        url = next_link

        # ウェイトを入れる（サーバーに負荷をかけないため）
        time.sleep(2)

    return items_data

def contains_excluded_keywords(blog_url):
    # ヘッダー情報をチェック
    if contains_excluded_keywords_in_header(blog_url):
        return True

    # プロフィール情報をチェック
    profile_url = f"https://www.ameba.jp/profile/{get_user_name(blog_url)}/"
    if contains_excluded_keywords_in_profile(profile_url):
        return True

    return False

def contains_excluded_keywords_in_header(blog_url):
    try:
        response = requests.get(blog_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # ヘッダー情報を取得
        header = soup.find('div', {'class': ['skin-headerTitle', 'skinBlogHeadingGroupArea']})
        header_text = header.get_text(strip=True) if header else ''

        # キーワードが含まれているかチェック
        for keyword in exclude_keywords:
            if keyword in header_text:
                return True
        return False

    except requests.exceptions.RequestException as e:
        print(f"{blog_url} からヘッダー情報を取得中にエラーが発生しました: {e}")
        return False

def contains_excluded_keywords_in_profile(profile_url):
    try:
        response = requests.get(profile_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # プロフィール情報を取得
        profile = soup.find('div', {'class': 'user-info'})
        profile_text = profile.get_text(strip=True) if profile else ''

        # キーワードが含まれているかチェック
        for keyword in exclude_keywords:
            if keyword in profile_text:
                return True
        return False

    except requests.exceptions.RequestException as e:
        print(f"{profile_url} からプロフィール情報を取得中にエラーが発生しました: {e}")
        return False

def scrape_blog_series(title, blog_url, max_blog_pages):
    blog_series_info = []
    page_num = 1
    last_subtitle = None
    last_content = None

    while page_num <= max_blog_pages:
        url = f"{blog_url}page-{page_num}.html" if page_num > 1 else blog_url
        blog_info = scrape_blog_info(title, url)
        
        current_subtitle = blog_info.get('title', 'N/A')
        current_content = blog_info.get('text', 'N/A')
        
        if current_subtitle == last_subtitle and current_content == last_content:
            break  # 前のページと同じサブタイトルと本文が出てきたら終了
        
        if current_subtitle == 'N/A' or current_content == 'N/A':
            break  # 終了条件：ブログ情報が見つからない場合

        blog_series_info.append(blog_info)
        last_subtitle = current_subtitle
        last_content = current_content
        page_num += 1
        time.sleep(2)  # サーバーに負荷をかけないためのウェイト

    return blog_series_info

def scrape_blog_info(title, blog_url):
    try:
        response = requests.get(blog_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # ブログのタイトルを取得
        if soup.find('div', class_='skinArticleHeader2'):  # div の場合
            blog_title_tag = soup.find('div', class_='skinArticleHeader2').find('h1').find('a')
            blog_title = blog_title_tag.get_text(strip=True) if blog_title_tag else 'N/A'
        elif soup.find('h2', class_='skin-entryTitle'):  # h2 の場合
            blog_title_tag = soup.find('h2', class_='skin-entryTitle').find('a', class_='skinArticleTitle')
            blog_title = blog_title_tag.get_text(strip=True) if blog_title_tag else 'N/A'
        elif soup.find('h1', class_='skinArticleTitle'):  # h1 の場合
            blog_title_tag = soup.find('h1', class_='skinArticleTitle').find('a', class_='skinArticleTitle')
            blog_title = blog_title_tag.get_text(strip=True) if blog_title_tag else 'N/A'
        else:
            blog_title = 'N/A'

        # ブログの投稿日を取得
        post_date = scrape_blog_post_date(blog_url)

        # ブログ本文を取得
        blog_content = scrape_blog_content(blog_url)

        # プロフィール情報を取得
        profile_info = scrape_profile_info(blog_url)
        # ブログの情報を辞書として返す
        blog_info = {
            'blog_title': title,
            'sex': profile_info['profile_gender'],
            'age': profile_info['profile_age'], 
            'user': profile_info['user_name'], 
            'url': blog_url, 
            'date': post_date,
            'title': blog_title,
            'text': blog_content
        }

    except requests.exceptions.RequestException as e:
        print(f"{blog_url} からブログ情報を取得中にエラーが発生しました: {e}")
        blog_info = {
            'blog_title': title,
            'sex': 'N/A',
            'age': 'N/A',
            'user': 'N/A', 
            'url': blog_url,
            'date': 'N/A', 
            'title': 'N/A',
            'text': 'N/A',
        }

    return blog_info

def scrape_blog_post_date(blog_url):
    try:
        response = requests.get(blog_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # ブログの投稿日時を取得
        if soup.find('time', class_='skin-textQuiet'):
            post_date_element = soup.find('time', class_='skin-textQuiet')
            if post_date_element.has_attr('datetime'):
                return post_date_element['datetime'].split('T')[0]

        # 'entry_created_datetime'が含まれるテキストを検索して日付を抽出
        elements_with_text = soup.find_all(string=re.compile(r'entry_created_datetime'))
        for element_text in elements_with_text:
            match = re.search(r'entry_created_datetime":"(\d{4}-\d{2}-\d{2})', element_text)
            if match:
                return match.group(1)

        return 'N/A'

    except requests.exceptions.RequestException as e:
        print(f"{blog_url} から投稿日を取得中にエラーが発生しました: {e}")
        return 'N/A'

def scrape_blog_content(blog_url):
    try:
        response = requests.get(blog_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # ブログ本文を含む要素を特定する（例として、指定されたHTMLを参照）
        blog_content = soup.find('div', {'data-google-interstitial': 'false', 'class': ['articleText', '_2nkwn0s9']})

        if blog_content:
            # ブログ本文を抜き出す
            blog_text = blog_content.get_text(separator=' ').strip()  # 改行せずに一行で表示
            return blog_text

        else:
            return 'N/A'

    except requests.exceptions.RequestException as e:
        print(f"{blog_url} からブログ本文を取得中にエラーが発生しました: {e}")
        return 'N/A'

def scrape_profile_info(blog_url):
    try:
        # プロフィールページのURLを構築
        profile_url = f"https://www.ameba.jp/profile/{get_user_name(blog_url)}/"
        
        # プロフィールページを取得
        response = requests.get(profile_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # プロフィール情報を取得
        profile_element = soup.find('div', class_='user-info__detail -clearfix')

        # 要素が存在するかを確認
        if profile_element:
            # プロフィール情報をリストに分割
            profile_info = profile_element.find_all('dl', class_='user-info__list')
            
            # 性別と生年月日の情報を初期化
            gender = "NaN"
            birth_date = None  # 生年月日を初期化

            # プロフィール情報から性別と生年月日を抽出
            for info in profile_info:
                term = info.find('dt', class_='user-info__term').text.strip()
                value = info.find('dd', class_='user-info__value').text.strip()
                if term == "性別":
                    gender = value
                elif term == "生年月日":
                    birth_date = datetime.strptime(value, "%Y年%m月%d日")

            # 年齢を計算
            if birth_date:
                today = datetime.now()
                age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
            else:
                age = "NaN"

        else:
            gender = "NaN"
            age = "NaN"

        return {'profile_gender': gender, 'profile_age': age, 'user_name': get_user_name(blog_url)}

    except requests.exceptions.RequestException as e:
        print(f"{profile_url} からプロフィール情報を取得中にエラーが発生しました: {e}")
        return {'profile_gender': 'N/A', 'profile_age': 'N/A', 'user_name': get_user_name(blog_url)}

def get_user_name(blog_url):
    # ブログのURLからユーザ名を抽出する関数
    return blog_url.split('/')[3]

# ランキングページのURL
url = 'https://blogger.ameba.jp/genres/diet/blogs/ranking'
items = scrape_ranking_items(url, max_pages)

# 各ブログの情報を取得して表示
if items:
    for item in items:
        print(f"ブログタイトル: {item['blog_title']}")
        print(f"リンク: {item['url']}")
        print(f"タイトル: {item['title']}")
        print(f"投稿日: {item['date']}")
        print(f"ブログ内容: {item['text'][:50]}...")  # 本文の先頭部分を表示（50文字まで）
        print(f"プロフィール性別: {item['sex']}")
        print(f"プロフィール年齢: {item['age']}")
        print(f"ユーザ名: {item['user']}")
        print('-' * 50)
else:
    print("要素が見つかりませんでした。")

ページ 1 を取得中...


ブログ取得中: 100%|██████████| 20/20 [00:49<00:00,  2.46s/it]


ページ 2 を取得中...


ブログ取得中: 100%|██████████| 20/20 [00:42<00:00,  2.10s/it]


ブログタイトル: 腹筋運動なんてしなくてよろし♡意識だけ美姿勢ダイエット♡勝手に痩せる体になる秘密
リンク: https://ameblo.jp/precious-being-day/
タイトル: 【一番人気記事】首の長さは姿勢で４cmも変わります
投稿日: 2024-07-18
ブログ内容: こんにちは 美姿勢インストラクター新田仁美です     「最少の動きで最大の効果」 が出る 姿勢改善...
プロフィール性別: NaN
プロフィール年齢: NaN
ユーザ名: precious-being-day
--------------------------------------------------
ブログタイトル: アラフォーボブ子の16時間断食オートファジーダイエット
リンク: https://ameblo.jp/bobdiet/
タイトル: ファスティング後に気をつけてること。　【ワンデイクレンズダイエット　その後】
投稿日: 2024-07-18
ブログ内容: こんにちは、ボブ子です こちらのブログはわたしボブ子が ハタチの頃の体重に戻すべく奮闘するブログです...
プロフィール性別: 女性
プロフィール年齢: NaN
ユーザ名: bobdiet
--------------------------------------------------
ブログタイトル: MONAの産後ダイエットで産前より愛されBODYに
リンク: https://ameblo.jp/mona163cmdiet/
タイトル: 【no整形】で小顔になれた美顔器 フェイスポインター corefit
投稿日: 2024-07-17
ブログ内容: 東京都内開催 　９月３日（火）・４（水）出張サロンします！ ＊場所：東京都　恵比寿駅近く場所の詳細は...
プロフィール性別: 女性
プロフィール年齢: NaN
ユーザ名: mona163cmdiet
--------------------------------------------------
ブログタイトル: なないろの➰28.3kg痩せた生活内容☘
リンク: https://ameblo.jp/nanairo2021/
タイトル: ダイエッターの【嫌な事!!】
投稿日: 2024-07-17
ブログ内容: 🌈にほんブログ村🌈 4つ

In [21]:
len(items)

31

In [27]:
items[3]

{'blog_title': 'なないろの➰28.3kg痩せた生活内容☘',
 'sex': '女性',
 'age': 47,
 'user': 'nanairo2021',
 'url': 'https://ameblo.jp/nanairo2021/',
 'date': '2024-07-17',
 'title': 'ダイエッターの【嫌な事!!】',
 'text': '🌈にほんブログ村🌈 4つ 押してくらさい 🙏 にほんブログ村 にほんブログ村 にほんブログ村 にほんブログ村     🌈 ご訪問ありがとうございます 🌈 人生の最小値\u3000 57.4kg🩷 いいね👍コメント大歓迎です🩷 🌈ありがとうございます🌈 身長163cm 2020.9.6と9.18二回尿管結石 恐らく86kg→ 57.3kg −28.8kg達成✨ MAX103kgからだと −45.7kg達成✨ AB韓方クリニック様アメブロ \xa0 『【価格改定のお知らせ】』 \xa0\xa0\xa0\xa0みなさんこんにちは韓国ABクリニックです\xa0\xa0\xa0\xa0\xa0この度お客様のご意見を参考に、ウォン→円表記に変更し価格を全体的に改定いたしました。\xa0\xa0今後はレート… ameblo.jp AB韓方クリニック様 では スタンプ制度 があるそうです。 ◆紹介されたご友人様への特典もございます◆ 紹介コードを提示されたお客様には購入月数分のスタンプ＋無条件にスタンプ1個サービス ◆共通事項◆ ・スタンプ5個でダイエット韓方のAB丸（1ヶ月分）が無料になります。 ※5個 貯まった次の購入時に プレゼントになります。 ・AB茶、便秘薬、睡眠丸のみのご購入や10日間お試しセットは対象外になります。 紹介コード 【407002】 お互いWIN-WIN♪ なので本気で痩せたくてご購入考えてる方 紹介コード お使い下さい🩷 \xa0 2024.7.10より チャレンジ開始!! AB丸チャレンジプロローグ \xa0 『新チャレンジ開始!!★プロローグ★』 🌈にほんブログ村🌈4つ押してくらさい🙏にほんブログ村にほんブログ村にほんブログ村にほんブログ村  🌈ご訪問ありがとうございます🌈人生初の最小値\u300057.7kg🩷… ameblo.jp 一読お願いします 感じた副作用 慣らし

In [1]:
# sex	age	user	url	date	title	text
# csvファイルを開く
import pandas as pd

df = pd.read_csv('diet_blogs_new4.csv')
df.tail()

,blog_title,sex,age,user,url,date,title,text
71253,ナツのココロオドル,女性,NaN,lilylife35,https://ameblo.jp/lilylife35/page-1466.html,2019-08-19,もしや！食欲の秋?!,こんにちは！ 今日からお仕事です。 なんとなくのんびりモード。 お兄ちゃんが保育園に行きたが...
71254,ナツのココロオドル,女性,NaN,lilylife35,https://ameblo.jp/lilylife35/page-1467.html,2019-08-18,断捨離と収納,こんにちは！ お盆休みも終わり。 明日から平常運転です。 先日ニトリで収納グッズを購入しまし...
71255,ナツのココロオドル,女性,NaN,lilylife35,https://ameblo.jp/lilylife35/page-1468.html,2019-08-16,貯蓄の考え方,こんにちは！ 今日はちょっとだけ仕事に行ってきました。 職場ではみんな現場で私一人。 お盆休...
71256,ナツのココロオドル,女性,NaN,lilylife35,https://ameblo.jp/lilylife35/page-1469.html,2019-08-15,小銭貯金のススメ,こんにちは！ 今日は弟くんを母に預けてお兄ちゃんと図書館に行ってきました。 昆虫ブームのお兄...
71257,ナツのココロオドル,女性,NaN,lilylife35,https://ameblo.jp/lilylife35/page-1470.html,2019-08-14,はじめまして,こんにちは！ ナツのブログにご訪問ありがとうございます。 サンキュ！の公式ブロガーとして5年...


In [2]:
df.head()

,blog_title,sex,age,user,url,date,title,text
0,腹筋運動なんてしなくてよろし♡意識だけ美姿勢ダイエット♡勝手に痩せる体になる秘密,NaN,NaN,precious-being-day,https://ameblo.jp/precious-being-day/,2024-07-18,【一番人気記事】首の長さは姿勢で４cmも変わります,こんにちは 美姿勢インストラクター新田仁美です 「最少の動きで最大の効果」 が出る ...
1,腹筋運動なんてしなくてよろし♡意識だけ美姿勢ダイエット♡勝手に痩せる体になる秘密,NaN,NaN,precious-being-day,https://ameblo.jp/precious-being-day/page-2.html,2024-07-17,口角を上げて印象アップ…それ以前に大事なこと２つ,こんにちは 美姿勢インストラクター新田仁美です。 「 最少の動きで最大の効果 」が出...
2,アラフォーボブ子の16時間断食オートファジーダイエット,女性,NaN,bobdiet,https://ameblo.jp/bobdiet/,2024-07-18,ファスティング後に気をつけてること。 【ワンデイクレンズダイエット その後】,こんにちは、ボブ子です こちらのブログはわたしボブ子が ハタチの頃の体重に戻すべく奮闘するブ...
3,アラフォーボブ子の16時間断食オートファジーダイエット,女性,NaN,bobdiet,https://ameblo.jp/bobdiet/page-2.html,2024-07-17,断食ダイエットが成功した秘訣 【ワンデイクレンズ 口コミ】,こんにちは、ボブ子です こちらのブログはわたしボブ子が ハタチの頃の体重に戻すべく奮闘するブ...
4,MONAの産後ダイエットで産前より愛されBODYに,女性,NaN,mona163cmdiet,https://ameblo.jp/mona163cmdiet/,2024-07-17,【no整形】で小顔になれた美顔器 フェイスポインター corefit,東京都内開催 ９月３日（火）・４（水）出張サロンします！ ＊場所：東京都 恵比寿駅近く場所...


In [12]:
# userが""precious-being-day"の行を抽出
df[df['url'] == 'https://ameblo.jp/tabetehappydiet/']

,blog_title,sex,age,user,url,date,title,text
49649,40代からお腹痩せ！美腹ンスダイエット（ビバランス）さいとうよしみ【東京】,女性,55.0,tabetehappydiet,https://ameblo.jp/tabetehappydiet/,2024-08-02,体重がするっと落ちた日の食事,万年ダイエッターで \n 何をしても痩せなかった私が、 \n \n 食べる組み合わせを変...
54766,40代からお腹痩せ！美腹ンスダイエット（ビバランス）さいとうよしみ【東京】,女性,55.0,tabetehappydiet,https://ameblo.jp/tabetehappydiet/,2024-08-02,体重がするっと落ちた日の食事,万年ダイエッターで \n 何をしても痩せなかった私が、 \n \n 食べる組み合わせを変...


In [5]:
# ユーザの一覧を取得
users = df['user'].unique()
users

array(['precious-being-day', 'bobdiet', 'mona163cmdiet', 'nanairo2021',
       'happybeauty0628', 'saesae2020', 'pumix777', 'akemi-4',
       'machiko-2kids', 'amelia-diet', 'yuki194', 'sweetjdog2002',
       'doraqu', 'usagi-202309', 'you0507diet', 'racoraco75',
       'komachi-diet', 'hrmamadiet', 'akibaboxinggym', 'avocadoforme',
       'miyu-725', 'aapinpin-580804', 'marty2367', 'tonton-diet-0320',
       'kumiko-rabu', 'kanaemon888', 'sakura2635', 'chisataxi',
       'sachi-dahlia1515', 'yurutabe', 'yukitukuyakukaemino', '428-diet',
       'nori-happylife-happymind', 'chico89915', 'teshigoto44',
       'watashino-dietblog', 'onnetu', 'mugitya07', '1116jan',
       'kimi-boku-w-2021', 'zhenxiataitai', 'tabeyase-diet-yuko',
       'nicky-dietdiary', 'akiring626', 'mami-sora-2023',
       'olmpia-serena4011', 'otomegogoroy', 'utsaxzxv', 'honey12130221',
       'mukoujima634', 'kanakramakid', 'kerokero0512', 'mm-diet-mm',
       'okome-rinatan', 'akdiet', 'remuera-nz', 'yukkosensei',


In [6]:
len(users)

80

In [7]:
# ユーザごとのブログ数を取得
user_counts = df['user'].value_counts()
user_counts

user
tiara55happy         7900
doraqu               5685
tabetehappydiet      4926
happybeauty0628      4207
utsaxzxv             4015
                     ... 
olmpia-serena4011       2
sakura2635              2
kumiko-rabu             2
avocadoforme            2
428-diet                2
Name: count, Length: 80, dtype: int64

In [8]:
len(user_counts)

57